In [ ]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import tensorflow as tf
import time

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 16*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

import sys
import os
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
import copy
import trimesh
%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook


In [ ]:
#load HD Map
pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
HD_map = trimesh.load(pl).vertices
show_nth = 5 #10
submap = HD_map[::show_nth]
plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(submap, c = "#CB2314", r = 2, alpha = 0.1)) 
plt.show(disp, "HD Map")
ViewInteractiveWidget(plt.window)

In [ ]:
#load ground truth
# [sec,nsec,x,y,z,qx,qy,qz,qw]
fn_gt = "/media/derm/06EF-127D4/Newer College Dataset/05_Quad_With_Dynamics/registered_poses.csv"
gt = np.loadtxt(fn_gt, delimiter=',',skiprows = 1)
print(len(gt))
seconds = gt[:, 0]
nano_seconds = gt[:, 1]
xyz = gt[:, 2:5]
qxyzw = gt[:, 5:]
num_poses = qxyzw.shape[0]
poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
poses[:, :3, :3] = R.from_quat(qxyzw).as_matrix()
poses[:, :3, 3] = xyz
T_CL = np.eye(4, dtype=np.float32)
T_CL[:3, :3] = R.from_quat([0.0, 0.0, 0.924, 0.383]).as_matrix() #was this --1134.97 deg
T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32) #was this
poses = np.einsum("nij,jk->nik", poses, T_CL)
initial_pose = np.linalg.inv(poses[0]) 
poses_timestamps = seconds * 10e9 + nano_seconds
poses = np.einsum("ij,njk->nik", np.linalg.inv(poses[0]), poses)

it = ICET(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 50, niter = 10, 
               draw = False, group = 2, RM = False, DNN_filter = False)
# it = ICET(cloud1 = submap, cloud2 = pc1, fid = 50, niter = 10, 
#                draw = False, group = 2, RM = False, DNN_filter = False)

In [ ]:
#put HD Map and new scan in frame of raw keyframe scan
# idx = 2700 #2750 
idx = 2500 #test
skip = 0 #10 #how many lidar frames between keyframe and new scan
# offset = 10 #works best for 05 dataset
offset = 10
fn1 = "/media/derm/06EF-127D4/Newer College Dataset/05_Quad_With_Dynamics/point_clouds_test/frame_" + str(idx + offset) + ".npy"
fn2 = "/media/derm/06EF-127D4/Newer College Dataset/05_Quad_With_Dynamics/point_clouds_test/frame_" + str(idx + skip + offset) + ".npy"
pc1 = np.load(fn1)
pc2 = np.load(fn2)
plt = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

#transform everything to pc1 frame
pc2_in_pc1_frame = (np.linalg.pinv(poses[idx]) @ poses[idx+skip] @ np.append(pc2, np.ones([len(pc2),1]), axis=1).T).T
pc2_in_pc1_frame = pc2_in_pc1_frame[:,:3] #remove extra axis
submap_in_pc1_frame = (np.linalg.pinv(poses[idx]) @ initial_pose @ np.append(submap, np.ones([len(submap),1]), axis =1).T).T
submap_in_pc1_frame = submap_in_pc1_frame[:,:3]

#remove points from HD Map far outside FOV of sensor -----------------
phis = LC.c2s(LC, submap_in_pc1_frame)[:,2]
not_too_low = np.argwhere(phis > 3*np.pi/8)[:,0]
not_too_high = np.argwhere(phis < 5*np.pi/8)[:,0] #test
good_phis = np.intersect1d(not_too_high, not_too_low)
submap_in_pc1_frame = submap_in_pc1_frame[good_phis,:]
#---------------------------------------------------------------------


disp.append(Points(submap_in_pc1_frame, c = "black", r = 2, alpha = 0.1)) ##CB2314

# #remove 4th column and center first scan about origin
# scan1_lidarframe = scan1_mapframe[:,:3] - poses[idx][:3,3]
# scan2_lidarframe = scan2_mapframe[:,:3] - poses[idx][:3,3]
# submap_lidarframe = submap_mapframe[:,:3] - poses[idx][:3,3]

#remove point from pc1 that hit person holding sensor (likely hurting ICP metrics)
pc1_spherical = tf.cast(it.c2s(pc1), tf.float32)
pc1_too_close = tf.where(pc1_spherical[:,0] < 2. )[:,0]
pc1_not_too_close = tf.where(pc1_spherical[:,0] > 2. )[:,0]
print(len(pc1_too_close))
pc1 = pc1[pc1_not_too_close.numpy()]
# disp.append(Points(pc1[pc1_too_close.numpy()], c = 'black', r = 5)) #red

# DRAW SCANS 1, 2
# disp.append(Points(pc1, c = '#a65852', r = 3)) #red
# disp.append(Points(pc2_in_pc1_frame, c = '#2c7c94', r = 3)) #blue
color = 255*np.linspace(0,1,len(pc1))
cname = np.array([255-color, color, 255-color]).T.tolist()
disp.append(Points(pc1, c=cname,  r = 3.5, alpha =0.5))


plt.show(disp, "05 Quad With Dynamics Frame #" + str(idx))
ViewInteractiveWidget(plt.window)

In [ ]:
#run VICET on single frame
from linear_corrector import LC #VICET

A0 = np.array([0.01*np.random.randn(), 0.01*np.random.randn(), 0.01*np.random.randn(), 0, 0, 0,
               0, 0, 0, 0, 0, 0])
# A0 = np.array([0., 0., 0, 0, 0, 0,
#                0, 0, 0, 0, 0, 0])
max_buffer = 0.125 

dc = LC(cloud1 = submap_in_pc1_frame, cloud2 = np.flip(pc1, axis = 0), fid = 55, niter = 50, 
        draw = True, mnp = 20, RM = False, solver = '12_state', 
        max_buffer = max_buffer, A0 = A0)


# dc = LC(cloud1 = pc1, cloud2 = pc2_in_pc1_frame, fid = 60, niter = 50, 
#         draw = True, mnp = 50, RM = False, solver = '12_state', 
#         max_buffer = max_buffer, A0 = A0)

ViewInteractiveWidget(dc.plt.window)

In [ ]:
#run ICET
initial_guess = tf.constant([0.01*np.random.randn(),0.01*np.random.randn(), 0.01*np.random.randn(),0.,0.,0.])
it = ICET(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 50, niter = 10, 
            draw = False, group = 2, RM = False, DNN_filter = False, x0 = initial_guess)

In [ ]:
#test ICP registration on a single pair of frames
# icp_rigid_transform, _, _ = trimesh.registration.icp(pc1, pc2_in_pc1_frame) #scan to scan
icp_rigid_transform, transformed_pc1, cost = trimesh.registration.icp(pc1, submap_in_pc1_frame, 
                                                                   threshold = 1e-7, max_iterations=150) #scan to HD Map
icp_trans = icp_rigid_transform[:3,3]
icp_euls = R.from_matrix(icp_rigid_transform[:3,:3]).as_euler('xyz')
print(icp_trans, icp_euls)
print(cost)

plt = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

alph = 0.4
disp.append(Points(pc1, c = 'red', r =3, alpha = alph))
disp.append(Points(transformed_pc1, c = 'blue', r =3, alpha = alph))
# disp.append(Points(submap_in_pc1_frame, c = 'grey', r = 3, alpha = alph))
disp.append(Points(submap_in_pc1_frame, c = "black", r = 2, alpha = 0.1)) ##CB2314

plt.show(disp, "ICP Test")
ViewInteractiveWidget(plt.window)

# Run VICET, NDT, and  ICP on full Newer College "05: Quad With Dynamics"

In [ ]:
from linear_corrector import LC #VICET
st = time.time()

# #OG RA-L submission --- not enough data!
# runlen = 200
# # start_idx = 2700
# start_idx = 1700 

#full dataset
runlen = 3954
start_idx = 0

A_hist = np.zeros([runlen, 12]) #VICET
VICET_pred_stds = np.zeros([runlen,12])
X_hist = np.zeros([runlen, 6]) #ICET/ NDT
X_hist_ICP = np.zeros([runlen, 6]) #ICP
raw_CD_hist = np.zeros(runlen)
VICET_CD_hist = np.zeros(runlen)
rigid_ICET_CD_hist = np.zeros(runlen)
ICP_CD_hist = np.zeros(runlen)

for idx in range(runlen):
    
    print("\n \n Frame #", idx + start_idx,"at", time.time()-st,"sec \n \n")
    
    #     get point cloud and initial transform -------------------------
#     05
    offset = 10
    fn1 = "/media/derm/06EF-127D4/Newer College Dataset/05_Quad_With_Dynamics/point_clouds_test/frame_" + str(start_idx + idx + offset) + ".npy"
    pc1 = np.load(fn1)
    #remove point from pc1 that hit person holding sensor (likely hurting ICP metrics)
    pc1_spherical = tf.cast(it.c2s(pc1), tf.float32)
    pc1_not_too_close = tf.where(pc1_spherical[:,0] > 2. )[:,0]
    pc1 = pc1[pc1_not_too_close.numpy()]

    #transform to pc1 frame ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    submap_in_pc1_frame = (np.linalg.pinv(poses[start_idx + idx]) @ initial_pose @ np.append(submap, np.ones([len(submap),1]), axis =1).T).T
    submap_in_pc1_frame = submap_in_pc1_frame[:,:3] #remove extra axis
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    #remove points from HD Map far outside FOV of sensor -----------------
    phis = LC.c2s(LC, submap_in_pc1_frame)[:,2]
    not_too_low = np.argwhere(phis > 3*np.pi/8)[:,0]
    not_too_high = np.argwhere(phis < 5*np.pi/8)[:,0] #test
    good_phis = np.intersect1d(not_too_high, not_too_low)
    submap_in_pc1_frame = submap_in_pc1_frame[good_phis,:]
    #---------------------------------------------------------------------
    #remove point from pc1 that hit person holding sensor
    pc1_spherical = tf.cast(it.c2s(pc1), tf.float32)
    pc1_too_close = tf.where(pc1_spherical[:,0] < 2. )[:,0]
    pc1_not_too_close = tf.where(pc1_spherical[:,0] > 2. )[:,0]
    print(len(pc1_too_close))
    pc1 = pc1[pc1_not_too_close.numpy()]
    
    # Run ICP ------------------------------------------------------------
#     icp_rigid_transform, _, _ = trimesh.registration.icp(pc1, submap_in_pc1_frame) #scan to HD Map
    init = np.array([[1, 0, 0, 0.01*np.random.randn()],
                     [0.,1, 0, 0.01*np.random.randn()],
                     [0, 0, 1, 0.01*np.random.randn()],
                     [0, 0, 0, 1]])
    icp_rigid_transform, transformed_pc1, _ = trimesh.registration.icp(pc1, submap_in_pc1_frame, initial = init,
                                                                   threshold = 1e-7, max_iterations=50) #scan to HD Map
    icp_trans = icp_rigid_transform[:3,3]
    icp_euls = R.from_matrix(icp_rigid_transform[:3,:3]).as_euler('xyz')
    X_hist_ICP[idx,:] = np.append(icp_trans, icp_euls) 
    print(icp_trans, icp_euls)

    np.save("results/ICP/ICP_05_start_from_0_v2", X_hist_ICP)
    
    # Run VICET ----------------------------------------------------------
#     A0 = np.array([0., 0, 0, 0, 0, 0,
#                0, 0, 0, 0, 0, 0])
#     seed with noisy initial transform to make things fair
    A0 = np.array([0.01*np.random.randn(), 0.01*np.random.randn(), 0, 0, 0, 0,
               0, 0, 0, 0, 0, 0])
    
    # for experiment #1
    max_buffer = 0.125 #0.5 #was 2.5 for toy problem
#     dc = LC(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 50, niter = 25, 
#             draw = False, mnp = 25, RM = True, solver = '12_state', 
#             max_buffer = max_buffer, A0 = A0)
    dc = LC(cloud1 = submap_in_pc1_frame, cloud2 = np.flip(pc1, axis=0), 
        fid = 55, niter = 50, draw = False, mnp = 20, RM = False, solver = '12_state', 
        max_buffer = max_buffer, A0 = A0)
    A_hist[idx,:] = dc.A
    VICET_pred_stds[idx,:] = dc.pred_stds
    np.save("results/VICET/VICET_05_start_from_0_v2", A_hist)

#     #run ICET ------------------------------------------------------------
    initial_guess = tf.constant([0.01*np.random.randn(),0.01*np.random.randn(),0.,0.,0.,0.])
    it = ICET(cloud1 = submap_in_pc1_frame, cloud2 = pc1, fid = 55, niter = 15, 
           draw = False, group = 2, RM = False, DNN_filter = False, x0 = initial_guess)
    X_hist[idx,:] = it.X

    np.save("results/ICET/ICET_05_start_from_0_v2", X_hist)

In [ ]:
#v1 -- old datatset
#v2 -- fid = 55, offset = 10